In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("KMeans").getOrCreate()
dataset = spark.read.csv("hack_data.csv" , inferSchema=True , header=True)
dataset.printSchema()
dataset.show(5)

22/09/14 14:33:15 WARN Utils: Your hostname, TIGER03366 resolves to a loopback address: 127.0.1.1; using 172.27.154.128 instead (on interface eth0)
22/09/14 14:33:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 14:33:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)

+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|            Location|WPM_Typing_Speed|
+-----------------------+-----------------+---------------+-----------------+---------------+--------------------+----------------+
|                    8.0|           391.09|              1|             2.96|            7.0|            Slovenia|     

In [2]:
dataset.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [3]:
dataset.select("Location").show(5)

+--------------------+
|            Location|
+--------------------+
|            Slovenia|
|British Virgin Is...|
|             Tokelau|
|             Bolivia|
|                Iraq|
+--------------------+
only showing top 5 rows



In [4]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

string_assemble = StringIndexer(inputCol="Location" , outputCol="Location_Indexed")
stringIndexer_model = string_assemble.fit(dataset)
dataset_new = stringIndexer_model.transform(dataset)


In [6]:
dataset_new.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'Location_Indexed']

In [7]:
assembler = VectorAssembler(inputCols=["Session_Connection_Time" , 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location_Indexed',
 'WPM_Typing_Speed'] , outputCol = "features")

In [8]:
data = assembler.transform(dataset_new)
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed',
 'Location_Indexed',
 'features']

In [10]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features" , outputCol="Sacled_features")
scaler_model = scaler.fit(data)
final_data = scaler_model.transform(data).select("Sacled_features")

In [13]:
final_data.show(5)

+--------------------+
|     Sacled_features|
+--------------------+
|[0.56785108466505...|
|[1.41962771166263...|
|[2.20042295307707...|
|[0.14196277116626...|
|[1.41962771166263...|
+--------------------+
only showing top 5 rows



In [15]:
from pyspark.ml.clustering import KMeans
for i in range(2,4):
    kmeans = KMeans(k=i , featuresCol="Sacled_features")
    model = kmeans.fit(final_data)
    results = model.transform(final_data)
    results.groupBy("prediction").count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         1|   61|
|         2|  106|
|         0|  167|
+----------+-----+



22/09/14 19:26:53 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 1968426 ms exceeds timeout 120000 ms
22/09/14 19:26:53 WARN SparkContext: Killing executors is not supported by current scheduler.
